## TensorFlow Operations 
- Basic operations
- Tensor types
- Importing data
- Lazy loading

In [1]:
import tensorflow as tf

### Visualize it with TensorBoard 

In [ ]:
a = tf.constant(2)
b = tf.constant(3)
x = tf.add(a, b)

with tf.Session() as sess:
    print(sess.run(x))

In [ ]:
tf.reset_default_graph() # Clears the default graph stack and resets the global default graph.

a = tf.constant(2, name='asdf')
b = tf.constant(3, name='ddd')
x = tf.add(a, b, name='add')
print(a,b,x)

# Create the summary writer after graph deifnition and before running your session
writer = tf.summary.FileWriter('./graphs/lec02/name', tf.get_default_graph())
with tf.Session() as sess:
    print(sess.run(x))
writer.close()

In [ ]:
!tensorboard --logdir="./graphs/lec02/name" --port 6007

## Constants, Sequences, Variables, Ops 

In [ ]:
tf.reset_default_graph()

a = tf.constant([2, 2], name='vector')
b = tf.constant([[0, 1], [2, 3]], name='matrix')
x = tf.multiply(a, b, name='mul')
# Broadcasting similar to Numpy
with tf.Session() as sess:
    print(sess.run(x))

### Tensors filled with a specific value

In [ ]:
# Creates a tensor of shape and all elements will be zeros 
print(tf.zeros([2, 3], tf.int32))
with tf.Session() as sess:
    print(sess.run(tf.zeros(shape=[2, 3], dtype=tf.int32)))

In [ ]:
# Creates a tensor of shape and type as the input_tensor
input_tensor = [[0, 1], [2, 3], [4, 5]]
print(tf.zeros_like(input_tensor))
with tf.Session() as sess:
    print(sess.run(tf.zeros_like(tensor=input_tensor)))

In [ ]:
# Creates a tensor of shape and all elements will be ones
with tf.Session() as sess:
    print(sess.run(tf.ones_like(tensor=input_tensor)))

In [ ]:
# Creates a tensor of shape and all elements will be scalar value
with tf.Session() as sess:
    print(sess.run(tf.fill(dims=[3, 4], value=5)))

### Constants as sequences 

In [ ]:
# Generates values in an interval.
with tf.Session() as sess:
    print(sess.run(tf.lin_space(start=10.0, stop=16.0, num=5)))

In [ ]:
# Creates a sequence of numbers.
with tf.Session() as sess:
    print(sess.run(tf.range(3, 18, 3)))
    print(sess.run(tf.range(5)))

In [ ]:
# Tensor objects are not iterable 
for _ in tf.range(4):
    print(_)

### Wizard of Div

In [ ]:
tf.reset_default_graph()
a = tf.constant([2, 3], name='a')
b = tf.constant([[0, 1], [2, 3]], name='b')
with tf.Session() as sess:
    print(sess.run(tf.div(b, a)))
    print(sess.run(tf.divide(b, a)))
    print(sess.run(tf.truediv(b, a)))
    print(sess.run(tf.floordiv(b, a)))
#     print(sess.run(tf.realdiv(b, a))) # error
    print(sess.run(tf.truncatediv(b, a)))
    print(sess.run(tf.floor_div(b, a)))

### TF vs NP Data Type 

In [ ]:
import numpy as np
tf.int32 == np.int32

In [ ]:
with tf.Session() as sess:
    print(sess.run(tf.ones([2,3], np.float32)))

In [ ]:
a = tf.zeros([2,3], np.int32)
with tf.Session() as sess:
    print(type(a))
    print('==sess.run==')
    print(type(sess.run(a)))

## Variables 
- What's wrong with constants?
    + Constants are stored in the graph definition 
    + This makes loading graphs expensive when constants are big
    + Only use constants for primitive types 
    
- tf.constant is an op
- tf.Variable is a class with many ops

In [ ]:
tf.reset_default_graph()

In [ ]:
# Create variables with tf.get_variable
s = tf.get_variable("scalar", initializer=tf.constant(2))
m = tf.get_variable("matrix", initializer=tf.constant([[0, 1], [2,3]]))
w = tf.get_variable("big_matrix", shape=(784, 10), initializer=tf.zeros_initializer())

In [ ]:
with tf.Session() as sess:
    print(sess.run(tf.report_uninitialized_variables()))

In [ ]:
# Initialize only a subset of variables
with tf.Session() as sess:
    sess.run(tf.variables_initializer([s, m]))
    print(sess.run(tf.report_uninitialized_variables()))

In [ ]:
# Initialize a single variable
with tf.Session() as sess:
    sess.run(w.initializer)
    print(sess.run(tf.report_uninitialized_variables()))

In [ ]:
# Initailize all variables at once
with tf.Session() as sess:
    print(sess.run(tf.report_uninitialized_variables()))
    sess.run(tf.global_variables_initializer())
    print(sess.run(tf.report_uninitialized_variables()))

### Eval() a variable
- sess.run() a variable

In [ ]:
W = tf.Variable(tf.truncated_normal([700, 10]))
with tf.Session() as sess:
    sess.run(W.initializer)
    print(W)
    print('-'*30+'eval()'+'-'*30)
    print(W.eval()) 

### tf.Variable.assign()

In [ ]:
# Example1
tf.reset_default_graph()
W = tf.Variable(10)
W.assign(100) # Assigns a new value to the variable.
with tf.Session() as sess:
    sess.run(W.initializer)
    print(W.eval()) # 100, uh...nope it's 10 why?

In [ ]:
# W.assign(100) creates an assign op. 
# That op needs to be executed in as session to take effect 
tf.reset_default_graph()
W = tf.Variable(10)
assign_op = W.assign(100)
with tf.Session() as sess:
    sess.run(W.initializer)
    sess.run(assign_op)
    print(W.eval()) # sess.run(W)

### assign_add() and assign_sub()

In [ ]:
my_var = tf.Variable(10)

with tf.Session() as sess:
    sess.run(my_var.initializer)
    # increment by 10
    print('assign_add(): {}'.format(sess.run(my_var.assign_add(10))))
    # decrement by 2
    print('assign_sub(): {}'.format(sess.run(my_var.assign_sub(3))))

### Each session maintains its own copy of variables 

In [ ]:
W = tf.Variable(10)

sess1 = tf.Session()
sess2 = tf.Session()

sess1.run(W.initializer)
sess2.run(W.initializer)

print(sess1.run(W.assign_add(10)))
print(sess2.run(W.assign_sub(3)))

print(sess1.run(W.assign_add(100)))
print(sess2.run(W.assign_sub(30)))

sess1.close()
sess2.close()

### Control Dependencies
- tf.Graph.control_dependencies(control_inputs)
- Defines which ops should be run first

In [ ]:
# g have 5 ops: a, b, c, d, e
g = tf.get_default_graph()
with g.control_dependencies([a, b, c]):
    # 'd' and 'e' will only run after 'a', 'b', 'c' have executed
    d = ... 
    e = .. 

## Placeholder 
- TF program
    + Assemble a graph
    + Use a session to execute operations in the graph
- Assemble the graph first without knowing the values needed for computations
- placeholder: later supply their own data when need to execute the computation

In [ ]:
tf.reset_default_graph()

In [ ]:
# Create a placeholder for a vector of 3 elements, type tf.float32
with tf.variable_scope('soojung'):
    a = tf.placeholder(tf.float32, shape=[3], name='a')
    b = tf.constant([5, 5, 5], tf.float32, name='b')
    c = a + b

writer = tf.summary.FileWriter(logdir='./graphs/lec02/placeholder',
                               graph=tf.get_default_graph())

with tf.Session() as sess:
    print(sess.run(c, feed_dict={a: [1, 2, 3]})) # the tensor a is the key, not the string 'a'
writer.close()

In [ ]:
tf.get_default_graph().as_graph_def()

In [ ]:
# You can feed_dict any feedable tensor
# Placeholder is just a way to indicate that something must be fed
tf.reset_default_graph()
a = tf.add(2, 5)
b = tf.multiply(a, 3)

with tf.Session() as sess:
    print(sess.run(b))
    print(sess.run(b, feed_dict={a: 15}))

## The trap of lazy loading
- Lazy loading: Defer declaring/initializing an object untill it is loaded 

In [2]:
# Normal loading 
tf.reset_default_graph()
x = tf.get_variable(name='x', initializer=tf.constant(10))
y = tf.get_variable(name='y', initializer=tf.constant(20))
z = tf.add(x, y) # Node 'add' added once to the graph definition

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(logdir='./graphs/lec02/normal_loading', graph=tf.get_default_graph())
    
    for _ in range(3):
        sess.run(z)
    else:
        print(tf.get_default_graph().as_graph_def())
        writer.close()

node {
  name: "Const"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 10
      }
    }
  }
}
node {
  name: "x"
  op: "VariableV2"
  attr {
    key: "container"
    value {
      s: ""
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
  attr {
    key: "shared_name"
    value {
      s: ""
    }
  }
}
node {
  name: "x/Assign"
  op: "Assign"
  input: "x"
  input: "Const"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "_class"
    value {
      list {
        s: "loc:@x"
      }
    }
  }
  attr {
    key: "use_locking"
    value {
      b: true
    }
  }
  attr {
    key: "validate_shape"
    value {
      b: true
    }
  }
}
node {
  name: "x/read"
  op: "Identity"
  input: "x"
  attr {
    k

In [3]:
# Lazy loading
tf.reset_default_graph()
x = tf.get_variable(name='x', initializer=tf.constant(10))
y = tf.get_variable(name='y', initializer=tf.constant(20))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(logdir='./graphs/lec02/lazy_loading', graph=tf.get_default_graph())
    
    for _ in range(3):
        sess.run(tf.add(x, y)) # Node 'add' added 10 times to the graph definition
    else:
        print(tf.get_default_graph().as_graph_def())
        writer.close()

node {
  name: "Const"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 10
      }
    }
  }
}
node {
  name: "x"
  op: "VariableV2"
  attr {
    key: "container"
    value {
      s: ""
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
  attr {
    key: "shared_name"
    value {
      s: ""
    }
  }
}
node {
  name: "x/Assign"
  op: "Assign"
  input: "x"
  input: "Const"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "_class"
    value {
      list {
        s: "loc:@x"
      }
    }
  }
  attr {
    key: "use_locking"
    value {
      b: true
    }
  }
  attr {
    key: "validate_shape"
    value {
      b: true
    }
  }
}
node {
  name: "x/read"
  op: "Identity"
  input: "x"
  attr {
    k